In [112]:
%matplotlib inline

import pandas
import matplotlib.pyplot as plt
import numpy as np

# Scikit imports
from sklearn.datasets import make_classification, make_regression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score

# Overfitting notebook

Notebook for overfitting simulation and notes.

## Discrete labels

### `make_classification`

- clusters of points normally distributed with $\sigma=1$ about vertices of an `n_informative`-dimensional hypercube with sides of length 2*`class_sep`


### Create synthetic data

In [401]:
# features
n_informative = 4
n_redundant = 2
n_total_feats = 20

# samples
n_train = 100
n_test = 200

# class parameters
n_classes = 2
weights = None # defaults to 50/50 split

# output params
shuffle = True
random_state = 0
class_sep = 1
n_clusters_per_class = 4

X, y = make_classification(
                            n_samples = n_train + n_test,
                            n_features = n_total_feats,
                            n_informative = n_informative,
                            n_redundant = n_redundant,
                            n_classes = n_classes,
                            weights = weights,
                            shuffle = shuffle,
                            random_state = random_state,
                            class_sep = class_sep)

end_idx_0 = n_train // 2
start_idx_1 = (n_train // 2) + (n_test // 2)
end_idx_1 = n_train + (n_test // 2)
train_data = np.append(X[:end_idx_0, :], X[start_idx_1:end_idx_1, :], 0)
train_data = np.c_[train_data, np.append(y[:end_idx_0], y[start_idx_1:end_idx_1])]

test_data = np.append(X[end_idx_0:start_idx_1, :], X[end_idx_1:, :], 0)
test_data = np.c_[test_data, np.append(y[end_idx_0:start_idx_1], y[end_idx_1:])]

train_X = train_data[:, :-1]
train_y = train_data[:, -1]

test_X = test_data[:, :-1]
test_y = test_data[:, -1]

In [402]:
print(sum(train_y))
print(sum(test_y))

47.0
104.0


In [414]:
degrees = np.arange(1,9)


train_acc = []
test_proba = []
test_acc = []
pipelines = []
for deg in degrees:
    poly_features = PolynomialFeatures(degree=deg, 
                                       include_bias=False,
                                       interaction_only=True)
    lr_model = LogisticRegression(solver='lbfgs', 
                                  penalty='none', 
                                  #C=1000,
                                  random_state=random_state,
                                  n_jobs=-1)
    lr_pipeline = Pipeline([("polynomial_features", poly_features),
                           ("logistic_regression", lr_model)])
    
    lr_pipeline.fit(train_X.copy(), train_y.copy())
    
    pipelines.append(lr_pipeline)
    
    train_acc.append(accuracy_score(train_y, lr_pipeline.predict(train_X)))
    test_proba.append(np.mean(lr_pipeline.predict_proba(test_X)[:, 0]))
    test_acc.append(accuracy_score(test_y, lr_pipeline.predict(test_X)))
    print(deg)
    
print(test_acc)

1
2
3
4
5
6
7
8
[0.71, 0.845, 0.695, 0.635, 0.54, 0.53, 0.49, 0.515]


In [415]:
train_acc

[0.84, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

In [416]:
test_proba

[0.4975156411446495,
 0.5147213608980707,
 0.5203572343000207,
 0.5325925971670543,
 0.5128486797771346,
 0.516220334203848,
 0.5327722450695098,
 0.5244140952639429]

In [417]:
np.max(train_X)

7.59236389408978

In [423]:
no_label_X = np.random.multivariate_normal(np.random.rand((n_total_feats)), 
                                           cov=np.eye(n_total_feats),
                                           size=n_test)

In [424]:
test_proba = []
test_pred = []
for pipe in pipelines:
    pred_prob = pipe.predict_proba(no_label_X)
    #print(pred_prob.shape)
    #print(pred_prob[:5,:])
    test_proba.append(np.mean(pred_prob[:, 1]))
    test_pred.append(np.sum(pipe.predict(no_label_X)))

In [425]:
test_proba

[0.8791821831992749,
 0.44951614452253735,
 0.4969247489110623,
 0.48669858358926105,
 0.4604947046060801,
 0.48182724817254197,
 0.47941191581319686,
 0.451512202326811]

In [426]:
test_pred

[186.0, 90.0, 99.0, 95.0, 88.0, 98.0, 94.0, 79.0]

In [427]:
for pipe in pipelines:
    print(pipe['polynomial_features'].n_output_features_)

20
210
1350
6195
21699
60459
137979
263949


### Notes

- generate data from two separate distributions
- plots as $p$ increases
- train on 50/50 split
- plots of underfit, "fit," and overfit predictions as the distribution of out-of-sample data goes from 0 to 1 in terms of labels
- use `make_classification`
- use pipeline of polynomial features
- TODO do the out-of-sample people come from a different data distribution?